<a href="https://colab.research.google.com/github/MorningStarTM/brain_cancer_detection/blob/main/brain_tumor_classification_with_simple_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.utils import shuffle
from glob import glob
from tensorflow.keras.layers import*
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import*
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
#parameters
H, W = 224, 224
channel = 3
num_class = 3
batch_size = 16
class_names = ["glioma", "meningioma", "notumor", "pituitary"]

#Data pipeline

In [3]:
def load_data(path, split=0.1):
    images = shuffle(glob(os.path.join(path, "*", "*.jpg")))
    
    split_size = int(len(images) * split)
    #split the data
    train_data, valid_data = train_test_split(images, test_size=split_size, random_state=42)
    train_data, test_data = train_test_split(train_data, test_size=split_size, random_state=42)

    return train_data, valid_data, test_data 

In [4]:
def process_image(path):
    #decode the path
    path = path.decode()
    #read image
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    #resize the image
    image = cv2.resize(image, [224, 224])
    #scale the image
    image = image / 255.0
    #change the data type of image
    image = image.astype(np.float32)

    #labeling the image
    class_name = path.split("/")[-2]
    class_idx = class_names.index(class_name)
    class_idx = np.array(class_idx, dtype=np.int32)

    return image, class_idx

In [5]:
def parse(path):
    image, labels = tf.numpy_function(process_image, [path], (tf.float32, tf.int32))
    labels = tf.one_hot(labels, 4)
    image.set_shape([224, 224, 3])
    labels.set_shape(4)
  
    return image, labels

In [6]:
#tensorflow dataset
def tf_dataset(images, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((images))
    dataset = dataset.map(parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(8)
    return dataset

#Model Creation

In [7]:
inputs = Input((W, H, channel), batch_size)

In [8]:
conv_x = Conv2D(16, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(inputs)
conv_x = Dropout(0.1)(conv_x)
conv_x = MaxPooling2D((2,2), strides=(2,2))(conv_x)
conv_x = Conv2D(16, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(conv_x)

conv_x = Conv2D(32, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(conv_x)
conv_x = Dropout(0.1)(conv_x)
conv_x = MaxPooling2D((2,2), strides=(1,1))(conv_x)

conv_x = Conv2D(64, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(conv_x)
conv_x = Dropout(0.1)(conv_x)
conv_x = MaxPooling2D((2,2), strides=(2,2))(conv_x)

conv_x = Conv2D(128, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(conv_x)
conv_x = Dropout(0.1)(conv_x)
conv_x = MaxPooling2D((2,2), strides=(2,2))(conv_x)

conv_x = Conv2D(256, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(conv_x)
conv_x = Dropout(0.1)(conv_x)
conv_x = MaxPooling2D((2,2), strides=(2,2))(conv_x)
conv_x = Conv2D(256, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(conv_x)

flatten = Flatten()(conv_x)
outputs = Dense(4, activation='softmax')(flatten)

In [9]:
model = Model(inputs=[inputs], outputs=[outputs])

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(16, 224, 224, 3)]       0         
                                                                 
 conv2d (Conv2D)             (16, 224, 224, 16)        448       
                                                                 
 dropout (Dropout)           (16, 224, 224, 16)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (16, 112, 112, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (16, 112, 112, 16)        2320      
                                                                 
 conv2d_2 (Conv2D)           (16, 112, 112, 32)        4640      
                                                             

In [11]:
model_path = "/content/drive/MyDrive/CNN_Models/brain_tumor_model_1.h5"
csv_path = "/content/drive/MyDrive/Model CSV/brain_tumor_1.csv"

In [12]:
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    CSVLogger(csv_path),
    ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=5, min_lr=1e-7, verbose=1)
]

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
path = "/content/drive/MyDrive/DataSet/brain_cancer/Training"

In [15]:
x_train, x_valid, x_test = load_data(path)
print(f"Train:{len(x_train)} - Test:{len(x_test)} - Valid:{len(x_valid)}")

Train:4586 - Test:573 - Valid:573


In [16]:
train_df = tf_dataset(x_train)
test_df = tf_dataset(x_test)
valid_df = tf_dataset(x_valid)

In [17]:
model.fit(
    train_df,
    validation_data=valid_df,
    epochs=20,
    callbacks=callbacks
)

Epoch 1/20
286/287 [============================>.] - ETA: 11s - loss: 0.7204 - accuracy: 0.7227

InvalidArgumentError: ignored